In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import regex as re
import sidetable
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [18]:
df_da = pd.read_csv('data/df_da.csv', index_col = 'index')
pd.options.display.max_columns = None
df_da.sample(4)

,age,gender,country,education,role,prog_language,code_env,notebook_kaggle,notebook_colab,notebook_azure,notebook_paperspace,notebook_binder,notebook_codeocean,notebook_ibmwatsonstudio,notebook_amazonsagemaker,notebook_amazonemr,notebook_googlecloudnotebooks,notebook_googleclouddatalab,notebook_databricks,notebook_zeppelin,notebook_deepnote,notebook_observable,notebook_OTHER,viz,field,salary,learn_dp_MySQL,learn_dp_PostgreSQL,learn_dp_SQLite,learn_dp_Oracle,learn_dp_Mongo,learn_dp_Snowflake,learn_dp_IBMDd2,learn_dp_SQLServer,learn_dp_AzureSQL,learn_dp_AzureCosmos,learn_dp_Redshift,learn_dp_Aurora,learn_dp_RDS,learn_dp_DynamoDB,learn_dp_BigQuery,learn_dp_CloudSQL,learn_dp_Firestore,learn_dp_BigTable,learn_dp_Spanner,learn_dp_Other,dp_most_freq,BI_tools,share_plotlydash,share_streamlit,share_NBViewer,share_GitHub,share_Personal blog,share_Kaggle,share_Colab,share_Shiny,share_None,share_OTHER,courses_Coursera,courses_edX,courses_Kaggle,courses_DataCamp,courses_Fastai,courses_Udacity,courses_Udemy,courses_LinkedIn,courses_Cloud-certification,courses_University,courses_OTHER,media_Twitter,media_Email,media_Reddit,media_Kaggle,media_Forums,media_YouTube,media_Podcasts,media_Blogs,media_Journal,media_Slack,media_OTHER,cat_experience,cat_salary
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9410,22-24,Prefer not to say,India,Doctoral degree,Data Analyst,Python,None,NaN,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn",Academics/Education,"2,000-2,999",NaN,NaN,NaN,NaN,MongoDB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Kaggle (notebooks, forums, etc)",NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...",NaN,NaN,NaN,NaN,NaN,Junior,< 25K
7946,22-24,Woman,Pakistan,Master’s degree,Data Analyst,"Python, SQL","Visual Studio ,Visual Studio Code (VSCode) ,Py...",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn",Computers/Technology,$0-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MySQL,NaN,NaN,NaN,NaN,GitHub,Personal blog,Kaggle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DataCamp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...",NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,Junior,< 25K
9000,40-44,Man,Netherlands,Doctoral degree,Business Analyst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accounting/Finance,"90,000-99,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Microsoft Azure SQL Database,Microsoft Azure Cosmos DB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sin experiencia,50K - 100K
15826,40-44,Woman,Colombia,Master’s degree,Data Analyst,SQL,"Visual Studio Code (VSCode) ,Jupyter Notebook",NaN,Colab Notebooks,Azure Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Computers/Technology,"25,000-29,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Microsoft Azure SQL Database,Microsoft Azure Synapse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,Coursera,NaN,NaN,NaN,NaN,NaN,Udemy,NaN,NaN,University Courses (resulting in a university ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Senior+,25K - 50K


In [4]:
df_da.describe(include='object').T

,count,unique,top,freq
age,3269,11,25-29,788
gender,3269,5,Man,2538
country,3269,66,India,795
education,3269,7,Master’s degree,1514
role,3269,2,Data Analyst,2301
...,...,...,...,...
media_Journal,260,1,"Journal Publications (peer-reviewed journals, ...",260
media_Slack,258,1,"Slack Communities (ods.ai, kagglenoobs, etc)",258
media_OTHER,82,1,Other,82
cat_experience,3269,5,Junior,1881


In [5]:
df_da.stb.missing()

,missing,total,percent
notebook_codeocean,3247,3269,99.327011
notebook_paperspace,3246,3269,99.296421
notebook_deepnote,3243,3269,99.204650
notebook_zeppelin,3241,3269,99.143469
share_NBViewer,3241,3269,99.143469
...,...,...,...
role,0,3269,0.000000
education,0,3269,0.000000
country,0,3269,0.000000
cat_experience,0,3269,0.000000


In [ ]:
# creamos diferentes df con datos sociodemográficos propios para cada columna que nos interesa 
# y poder expandirlas sin crear demasiadas filas


sociodemo_language = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","prog_language"] ]
sociodemo_language["languages"] = sociodemo_language["prog_language"].str.split(",")
df_languages = sociodemo_language.explode(["languages"])
df_languages.drop("prog_language", inplace= True, axis= 1)
df_languages.sample(2)

In [42]:
sociodemo_code_env = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","code_env"] ]
sociodemo_code_env["cod_env"] = sociodemo_code_env["code_env"].str.split(",")
df_code_env = sociodemo_code_env.explode(["cod_env"])
df_code_env.drop("code_env", inplace= True, axis= 1)
df_code_env.sample(2)

/tmp/ipykernel_23990/993416549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sociodemo_code_env["cod_env"] = sociodemo_code_env["code_env"].str.split(",")


,age,gender,country,education,role,field,cat_experience,cat_salary,cod_env
index,,,,,,,,,
3236,22-24,Woman,India,Master’s degree,Data Analyst,Computers/Technology,Junior,< 25K,Jupyter Notebook
10065,18-21,Man,India,Bachelor’s degree,Data Analyst,Other,Junior,< 25K,Jupyter (JupyterLab


In [43]:
sociodemo_viz = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","viz"] ]
sociodemo_viz["visual"] = sociodemo_viz["viz"].str.split(",")
df_viz = sociodemo_viz.explode(["visual"])
df_viz.drop("viz", inplace= True, axis= 1)
df_viz.sample(2)

/tmp/ipykernel_23990/2257016201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sociodemo_viz["visual"] = sociodemo_viz["viz"].str.split(",")


,age,gender,country,education,role,field,cat_experience,cat_salary,visual
index,,,,,,,,,
17831,35-39,Man,United States of America,Master’s degree,Data Analyst,Academics/Education,Mid,100K - 200K,Shiny
19291,30-34,Man,"Iran, Islamic Republic of...",Master’s degree,Business Analyst,Computers/Technology,Junior,< 25K,Matplotlib


In [45]:
df_languages.to_csv("data/df_languages.csv")
df_viz.to_csv("data/df_viz.csv")
df_code_env.to_csv("data/df_code_env.csv")

In [ ]:
columnas_dividir = ["prog_language", "code_env", "viz"]

lista = []

for col in df_da[columnas_dividir]:
    lista.append(list(df_da[col].str.split(",")))

In [ ]:
df_da["test"] = df_da["prog_language"].str.split(",")
df_da.explode(["test"])

In [24]:
df_da["test"] = df_da["prog_language"].str.split(",")
df_da.explode(["test"])

,age,gender,country,education,role,prog_language,code_env,notebook_kaggle,notebook_colab,notebook_azure,notebook_paperspace,notebook_binder,notebook_codeocean,notebook_ibmwatsonstudio,notebook_amazonsagemaker,notebook_amazonemr,notebook_googlecloudnotebooks,notebook_googleclouddatalab,notebook_databricks,notebook_zeppelin,notebook_deepnote,notebook_observable,notebook_OTHER,viz,field,salary,learn_dp_MySQL,learn_dp_PostgreSQL,learn_dp_SQLite,learn_dp_Oracle,learn_dp_Mongo,learn_dp_Snowflake,learn_dp_IBMDd2,learn_dp_SQLServer,learn_dp_AzureSQL,learn_dp_AzureCosmos,learn_dp_Redshift,learn_dp_Aurora,learn_dp_RDS,learn_dp_DynamoDB,learn_dp_BigQuery,learn_dp_CloudSQL,learn_dp_Firestore,learn_dp_BigTable,learn_dp_Spanner,learn_dp_Other,dp_most_freq,BI_tools,share_plotlydash,share_streamlit,share_NBViewer,share_GitHub,share_Personal blog,share_Kaggle,share_Colab,share_Shiny,share_None,share_OTHER,courses_Coursera,courses_edX,courses_Kaggle,courses_DataCamp,courses_Fastai,courses_Udacity,courses_Udemy,courses_LinkedIn,courses_Cloud-certification,courses_University,courses_OTHER,media_Twitter,media_Email,media_Reddit,media_Kaggle,media_Forums,media_YouTube,media_Podcasts,media_Blogs,media_Journal,media_Slack,media_OTHER,cat_experience,cat_salary,test
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,"Python, SQL","Jupyter (JupyterLab, Jupyter Notebooks, etc) ,...",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Energy/Mining,"2,000-2,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Microsoft Azure SQL Database,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coursera,NaN,Kaggle Learn Courses,NaN,NaN,NaN,NaN,NaN,"Cloud-certification programs (direct from AWS,...",NaN,NaN,NaN,NaN,NaN,"Kaggle (notebooks, forums, etc)",NaN,NaN,NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,Senior+,< 25K,Python
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,"Python, SQL","Jupyter (JupyterLab, Jupyter Notebooks, etc) ,...",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Energy/Mining,"2,000-2,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Microsoft Azure SQL Database,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coursera,NaN,Kaggle Learn Courses,NaN,NaN,NaN,NaN,NaN,"Cloud-certification programs (direct from AWS,...",NaN,NaN,NaN,NaN,NaN,"Kaggle (notebooks, forums, etc)",NaN,NaN,NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,Senior+,< 25K,SQL
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,R,RStudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN,NaN,NaN,NaN,NaN,Ggplot / ggplot2,Manufacturing/Fabrication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Junior,NaN,R
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Python,"Notepad++,Jupyter Notebook",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Non-profit/Service,"7,500-9,999",MySQL,NaN,SQLite,NaN,MongoDB,NaN,NaN,Microsoft SQL Server,Microsoft Azure SQL Database,Microsoft Azure Cosmos DB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,Coursera,NaN,Kaggle Learn Courses,DataCamp,NaN,NaN,NaN,NaN,NaN,University Courses (resulting in a university ...,Other,Twitter (data science influencers),"Email newsletters (Data Elixir, O'Reilly Data ...",NaN,"Kaggle (notebooks, forums, etc)",NaN,"YouTube (Kaggle YouTube, Cloud AI Advent

In [ ]:
# df.explode("columna")

In [6]:
# creamos una lista con todas las columnas relacionadas a la pregunta notebooks

lista_notebooks = ['notebook_kaggle', 'notebook_colab', 'notebook_azure',
       'notebook_paperspace', 'notebook_binder', 'notebook_codeocean',
       'notebook_ibmwatsonstudio', 'notebook_amazonsagemaker',
       'notebook_amazonemr', 'notebook_googlecloudnotebooks',
       'notebook_googleclouddatalab', 'notebook_databricks',
       'notebook_zeppelin', 'notebook_deepnote', 'notebook_observable',
       'notebook_OTHER']

In [8]:
# creamos un df sólo para esa pregunta

df_notebooks = df_da[lista_notebooks]
df_notebooks.sample(3)

,notebook_kaggle,notebook_colab,notebook_azure,notebook_paperspace,notebook_binder,notebook_codeocean,notebook_ibmwatsonstudio,notebook_amazonsagemaker,notebook_amazonemr,notebook_googlecloudnotebooks,notebook_googleclouddatalab,notebook_databricks,notebook_zeppelin,notebook_deepnote,notebook_observable,notebook_OTHER
index,,,,,,,,,,,,,,,,
17444,Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16754,Kaggle Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22927,NaN,NaN,Azure Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
""" lista_valores = []

for col in df_notebooks.columns:
    for element in col:
        if element != np.nan:
            lista_valores.append(element)
"""

In [16]:
"""
def unir(valor):

    lista_valores= []

    for col in df_notebooks.columns:
        if valor != np.nan:
          lista_valores.append(f"{valor}, ")
    
    return lista_valores

"""

In [ ]:
# df_notebooks["prueba"]= df_notebooks[lista_notebooks].apply(unir)